In [1]:
from pandas_datareader import data as pdr 
from datetime import date
import yfinance as yf 
yf.pdr_override()
import pandas as pd
import matplotlib.pyplot as plt 
import math
import quandl
import numpy as np

/home/juan/.local/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
ticker_sp = '^GSPC'
ticker_gold = 'GC=F'
ticker_oil = 'CL=F'
ticker_dax = '^GDAXI'
ticker_nikkei = '^N225'
ticker_ftse = '^FTSE'
ticker_shanghai = '000001.SS'

auth_tok = "Nv1rJgRR7u88iz_dg7Y6"

end_date = "2020-06-1"
start_date = "2000-01-01"

In [3]:
def getGOLDData ():
    # Contains price and volume
    data = quandl.get("CHRIS/CME_GC1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[['Last']]
    data.columns = ["GOLD Adj Close"]
    return data

def getSPData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_sp, start=start_date, end=end_date)
    data = data[data.columns[4:6]] 
    data.columns = ["SP500 Adj Close",  "SP500 Volume"]
    return data

def getDAXData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_dax, start=start_date, end=end_date)
    data = data[data.columns[4:6]]
    data.columns = ["DAX Adj Close",  "DAX Volume"]
    return data


def getOILData():
    # Contains price and volume
    data = quandl.get("CHRIS/CME_CL1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[["Last"]]
    data.columns=["OIL Adj Close"]
    return data


def getNIKKEIData():
    # Contains only price
    data = pdr.get_data_yahoo(ticker_nikkei, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["NIKKEI Adj Close"]
    return data


def getFTSEData():
    # Contains price and volume
    data = pdr.get_data_yahoo(ticker_ftse, start=start_date, end=end_date)
    data = data[data.columns[4:6]] 
    data.columns = ["FTSE Adj Close",  "FTSE Volume"]
    return data

def getSHANGHAIData():
    # Contains only price
    data = pdr.get_data_yahoo(ticker_shanghai, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["SHANGHAI Adj Close"]
    return data

In [4]:
def combineData():
    allData = [getSPData(), getDAXData(), getFTSEData(), getGOLDData(), getOILData()]
    mergedData = pd.concat(allData, axis = 1)
    cleanData = mergedData.dropna()
    return cleanData

In [18]:
data = combineData()
print(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
            SP500 Adj Close  SP500 Volume  DAX Adj Close   DAX Volume  \
Date                                                                    
2000-01-04      1399.420044  1.009000e+09    6586.950195   46678400.0   
2000-01-05      1402.109985  1.085500e+09    6502.069824   52682800.0   
2000-01-06      1403.449951  1.092300e+09    6474.919922   41180600.0   
2000-01-07      1441.469971  1.225200e+09    6780.959961   56058900.0   
2000-01-10      1457.599976  1.064800e+09    6925.520020   42006200.0   
...                     ...           ...            ...          ...   
2020-05-15      2863.699951  5.477040e+09   10465.169922  116082600.0   
2020-05-18      2953.909912  6.364290e+09   11058.870117  118984900.0   
2020-05-19      2922.939941  4.969330e+09   11075.290039  12554

In [19]:
FUTURE_TO_PREDICT = 1

data['Future'] = data['SP500 Adj Close'].shift(-FUTURE_TO_PREDICT)

data.dropna(inplace=True)

In [20]:
def buy_or_sell (current, future):
    if (future > current):
        return 1
    else:
        return 0
    
data['target'] = list(map(buy_or_sell, data['SP500 Adj Close'], data['Future']))

In [26]:
NUMBER_OF_DATA_POINTS = len(data)
SIZE_TRAINING = int(NUMBER_OF_DATA_POINTS * 0.85)
SIZE_TESTING  = NUMBER_OF_DATA_POINTS - SIZE_TRAINING
print("size of training data: {}".format(SIZE_TRAINING))
print("size of testing data: {}".format(SIZE_TESTING))

data_training = data[:SIZE_TRAINING]
data_testing  = data[SIZE_TRAINING:]

size of training data: 4186
size of testing data: 739


In [29]:
print(data_training)

            SP500 Adj Close  SP500 Volume  DAX Adj Close   DAX Volume  \
Date                                                                    
2000-01-04      1399.420044  1.009000e+09    6586.950195   46678400.0   
2000-01-05      1402.109985  1.085500e+09    6502.069824   52682800.0   
2000-01-06      1403.449951  1.092300e+09    6474.919922   41180600.0   
2000-01-07      1441.469971  1.225200e+09    6780.959961   56058900.0   
2000-01-10      1457.599976  1.064800e+09    6925.520020   42006200.0   
...                     ...           ...            ...          ...   
2017-03-15      2385.260010  3.906840e+09   12009.870117  101358200.0   
2017-03-16      2381.379883  3.365660e+09   12083.179688  119337500.0   
2017-03-17      2378.250000  5.178040e+09   12095.240234  187435700.0   
2017-03-20      2373.469971  3.054930e+09   12052.900391  107350200.0   
2017-03-21      2344.020020  4.265590e+09   11962.129883  130893800.0   

            FTSE Adj Close   FTSE Volume  GOLD Adj

In [ ]:
def process_data(data):
    data.drop("Future", axis = 1, inplace = True)

            SP500 Adj Close  SP500 Volume  DAX Adj Close   DAX Volume  \
Date                                                                    
2000-01-04      1399.420044  1.009000e+09    6586.950195   46678400.0   
2000-01-05      1402.109985  1.085500e+09    6502.069824   52682800.0   
2000-01-06      1403.449951  1.092300e+09    6474.919922   41180600.0   
2000-01-07      1441.469971  1.225200e+09    6780.959961   56058900.0   
2000-01-10      1457.599976  1.064800e+09    6925.520020   42006200.0   
...                     ...           ...            ...          ...   
2020-05-14      2852.500000  5.641920e+09   10337.019531  133623600.0   
2020-05-15      2863.699951  5.477040e+09   10465.169922  116082600.0   
2020-05-18      2953.909912  6.364290e+09   11058.870117  118984900.0   
2020-05-19      2922.939941  4.969330e+09   11075.290039  125543400.0   
2020-05-20      2971.610107  4.992970e+09   11223.709961   98827200.0   

            FTSE Adj Close   FTSE Volume  GOLD Adj